In [2]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [3]:
# Define the path
#path = os.path.join("GitHub", "DataProcessing")

# Print the path
#print(path)

In [4]:
#pwd

In [5]:
#td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [6]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r") as file:
    postData = json.load(file)

In [7]:
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']
framesData = data['info']['frames']
participantFrames = [frame['participantFrames'] for frame in framesData]


postParticipantData = postData['info']['participants']
postChallengeData = [participants['challenges'] for participants in postParticipantData]

In [8]:
# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)
mostMentionedNamesDict = {}


In [9]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

    return championCountsDict, namesDict

# Calculate champion counts
championCountsDict, namesDict = calculateChampionCounts(data)

# Process the counts and choose the most mentioned champion name for each ID
mostMentionedNamesDict = {}
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    mostMentionedNamesDict[participantId] = mostMentionedChampion
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")

# Access the most mentioned champion name for a specific participant ID
# Example: print(mostMentionedNamesDict[1])
mostMentionedNamesDict


{8: 'Irelia',
 3: 'TwistedFate',
 7: 'Rengar',
 1: 'Sion',
 6: 'DrMundo',
 2: 'MasterYi',
 9: 'Vladimir',
 5: 'Senna',
 4: 'Caitlyn',
 10: 'Thresh'}

In [10]:
participantData = []

participantChampionMapping = {}

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

participantData[0]


[1,
 'RGZA7Bax07I4r0bt9cvYtwviMHqAHtDpxcHPsu-BOd7sdSv4uZ5dlsvMWtqcf2qlNdFw5-qzeLRMuQ',
 'Sion']

In [11]:
participantData

[[1,
  'RGZA7Bax07I4r0bt9cvYtwviMHqAHtDpxcHPsu-BOd7sdSv4uZ5dlsvMWtqcf2qlNdFw5-qzeLRMuQ',
  'Sion'],
 [2,
  '92u7ZxgOLcJuu0VAAcp86ucO0c8ITXLkCilyh7PvlLr3D-D_9MGkWRVNLkzdgU-_GRMQLV7WstNILQ',
  'MasterYi'],
 [3,
  'MDH3x-fDIsFOYVv-gMBJMDuM87d__pv8vqcOdpLdrpJ_Yv8XHrWmsVrx2HZNGdM0XdlOZGb93KAmUw',
  'TwistedFate'],
 [4,
  'RPcQMdOrtm2u3OW1R0clAWQHpJv2WVBQR2OI-marA9quYyx6vLM7KnkvSflHfi5gJlsqHT0PmxYYOw',
  'Caitlyn'],
 [5,
  'qqLzRit0Y8mcgNefznUaZUfIIny9LqJbFJ4tnRTdRDZ5rZliVKCJdK3DaEEKTtjU2MkSQTvGXDKarw',
  'Senna'],
 [6,
  'LMwDnwze39MQOJgt1E7qyTsyezolvZYzZcoKYkwUSH6wIRNhR_SBU8HnM0pqiScp9k3x6grcPhBL8g',
  'DrMundo'],
 [7,
  'cKZG1U1P8U1dxhVZOlcLXwA4lcQ-_rDGu2ezewd_G9TWQcgK9Bad1Gy6k2HtL1XZ6xmPaumNGAecvw',
  'Rengar'],
 [8,
  'MX203FDi8tibNVeWQ9wZuZTdHiwkRXtCgqfZLYtfgK40egA4rbV9GG1CsFLq1lDSz6pT9K2eFsTaiQ',
  'Irelia'],
 [9,
  'LrkpI6m_6xMJlggqwdX_SF24t2bpDK45FJyhAFDPER1k46Rz6wyDwiew3MCLB-H5nssiSjgvzXzPlQ',
  'Vladimir'],
 [10,
  'W-1DrDTs6jMaNKGFcEdtxU_dx67vreul3jY8xbUewZMjsRoSAsaa3kw9V5DHSM1Du

In [12]:
# Function to find participant data in kill events
def findParticipantDataInKillEvents(data):
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    
                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

# Example usage
findParticipantDataInKillEvents(data)

# Process the counts and choose the most mentioned champion name for each ID
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")


In [13]:
# Get every timestamp within a game paired with corresponding minute
def getTimestamps():
    for counter, frame in enumerate(data['info']['frames']):
        timestamp = frame['timestamp']
        print(f"{counter + 1} Timestamp = {timestamp}")

#getTimestamps()

In [14]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

In [15]:
def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key}:")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [16]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = "2"
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120031
championStats:
    armor: 36
    attackDamage: 71
    attackSpeed: 111
    ccReduction: 5
    health: 242
    healthMax: 763
    healthRegen: 23
    magicResist: 33
    movementSpeed: 355
    power: 261
    powerMax: 281
    powerRegen: 23
  currentGold: 161
damageStats:
    totalDamageDone: 3145
    totalDamageTaken: 949
  jungleMinionsKilled: 4
  level: 2
  participantId: 2
position:
    x: 7927
    y: 2312
  totalGold: 611
  xp: 325

---



In [17]:
def getChampionKilled(data, startTimestamp, endTimestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        if startTimestamp <= timestamp <= endTimestamp:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killTimestamp = event['timestamp']
                    print(f"\nTimestamp of CHAMPION_KILL event: {killTimestamp}")

                    # Get the champion name of the killed participant using victimId
                    victimId = event.get('victimId')
                    victimData = next((p for p in participantData if p[0] == victimId), None)
                    if victimData:
                        victimChampionName = victimData[2]
                        print(f"Champion killed: {victimChampionName} (ID: {victimId})")

                        print("Victim Damage Received:")
                        processedNames = set()
                        for damageInfo in event.get('victimDamageReceived', []):
                            name = damageInfo['name']
                            if name in [p[2] for p in participantData] and name not in processedNames:
                                print(f"  Name: {name}")
                                processedNames.add(name)
                    else:
                        print(f"No participant data found for victim ID: {victimId}\n")

# Example usage: Specify the start and end timestamps
startTimestamp = 120000  # Replace with your desired start timestamp
endTimestamp = 500000    # Replace with your desired end timestamp

# Call the function with the specified time frame
getChampionKilled(data, startTimestamp, endTimestamp)



Timestamp of CHAMPION_KILL event: 145207
Champion killed: TwistedFate (ID: 3)
Victim Damage Received:
  Name: Irelia

Timestamp of CHAMPION_KILL event: 145474
Champion killed: Irelia (ID: 8)
Victim Damage Received:
  Name: TwistedFate

Timestamp of CHAMPION_KILL event: 174151
Champion killed: Sion (ID: 1)
Victim Damage Received:
  Name: DrMundo
  Name: Rengar

Timestamp of CHAMPION_KILL event: 177421
Champion killed: Rengar (ID: 7)
Victim Damage Received:
  Name: Sion

Timestamp of CHAMPION_KILL event: 179220
Champion killed: DrMundo (ID: 6)
Victim Damage Received:
  Name: Sion

Timestamp of CHAMPION_KILL event: 216067
Champion killed: Irelia (ID: 8)
Victim Damage Received:
  Name: TwistedFate
  Name: MasterYi

Timestamp of CHAMPION_KILL event: 234287
Champion killed: Rengar (ID: 7)
Victim Damage Received:
  Name: MasterYi
  Name: TwistedFate

Timestamp of CHAMPION_KILL event: 253996
Champion killed: TwistedFate (ID: 3)
Victim Damage Received:
  Name: Irelia

Timestamp of CHAMPION_KIL

In [18]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def getEventOfChampionKillAtTimestamp(data, targetTimestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == targetTimestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
targetTimestamp = 234287
championKillEvent = getEventOfChampionKillAtTimestamp(data, targetTimestamp)

# Print or use the champion_kill_event data as needed
if championKillEvent:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(championKillEvent)
else:
    print(f"No CHAMPION_KILL event found at timestamp {targetTimestamp}.")


CHAMPION_KILL event data:
{'assistingParticipantIds': [3],
 'bounty': 300,
 'killStreakLength': 1,
 'killerId': 2,
 'position': {'x': 7413, 'y': 7943},
 'shutdownBounty': 0,
 'timestamp': 234287,
 'type': 'CHAMPION_KILL',
 'victimDamageDealt': [{'basic': False,
                        'magicDamage': 25,
                        'name': 'Rengar',
                        'participantId': 2,
                        'physicalDamage': 0,
                        'spellName': '',
                        'spellSlot': -1,
                        'trueDamage': 14,
                        'type': 'OTHER'},
                       {'basic': False,
                        'magicDamage': 0,
                        'name': 'Rengar',
                        'participantId': 2,
                        'physicalDamage': 54,
                        'spellName': 'rengare',
                        'spellSlot': 2,
                        'trueDamage': 0,
                        'type': 'OTHER'},
             

In [37]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = event['timestamp']
                        victimDamageReceived = event.get('victimDamageReceived', [])
                        killPosition = event.get('position', {})  # Added line

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists,
                            'position': killPosition  # Added line
                        })

    return killData

# Example usage:
participantIdToCheck = 1  # Replace with the desired participant ID
killDataForParticipant = getKillDataForParticipant(data, participantIdToCheck, participantData)

# Display the kill data with total damage done, assisting champions, and damage by assists
for kill in killDataForParticipant:
    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""

    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}"
          f"Kill Location: (X: {kill['position'].get('x', 0)}, Y: {kill['position'].get('y', 0)})\n")  # Added line


Timestamp: 177421
Killer: Sion (ID: 1)
Victim: Rengar (ID: 7)
Total Damage Done: 335
Kill Location: (X: 4258, Y: 13593)

Timestamp: 179220
Killer: Sion (ID: 1)
Victim: DrMundo (ID: 6)
Total Damage Done: 815
Kill Location: (X: 3420, Y: 13841)

Timestamp: 1022820
Killer: Sion (ID: 1)
Victim: DrMundo (ID: 6)
Total Damage Done: 727
Assisting Champions: MasterYi, Caitlyn, Senna.
Damage Done by Assists: 1738 by MasterYi, 0 by Caitlyn, 0 by Senna.
Kill Location: (X: 1684, Y: 13541)

Timestamp: 1571353
Killer: Sion (ID: 1)
Victim: Rengar (ID: 7)
Total Damage Done: 632
Kill Location: (X: 8185, Y: 11808)

Timestamp: 1877884
Killer: Sion (ID: 1)
Victim: Rengar (ID: 7)
Total Damage Done: 569
Assisting Champions: MasterYi, TwistedFate.
Damage Done by Assists: 1352 by MasterYi, 695 by TwistedFate.
Kill Location: (X: 10292, Y: 9954)



In [40]:
def getAssistsForParticipant(data, targetParticipantId, participantData):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = event['timestamp']
                        assistName = next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                          f"Unknown Champion with ID {targetParticipantId}")

                        assistData.append({
                            'timestamp': timestamp,
                            'assistId': targetParticipantId,
                            'assistName': assistName,
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })

    return assistData

# Example usage:
targetParticipantIdToCheck = 1  # Replace with the desired target participant ID
assistDataForParticipant = getAssistsForParticipant(data, targetParticipantIdToCheck, participantData)

# Display the assist data with timestamps and coordinates
for assist in assistDataForParticipant:
    print(f"Timestamp: {assist['timestamp']}\n"
          f"Assist: {assist['assistName']} (ID: {assist['assistId']})\n"
          f"Assist Location: (X: {assist['position'].get('x', 0)}, Y: {assist['position'].get('y', 0)})\n")


NameError: name 'targetParticipantId' is not defined

In [20]:
def getTimestampOfChampionKillWithId(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')
                victimNames = [victim.get('name') for victim in event.get('victimDamageDealt', [])]
                killerNames = [killer.get('name') for killer in event.get('victimDamageReceived', [])]

                if victimNames:  # Check if there are victim names
                    victimName = victimNames[0]
                    print(f"{victimName} was killed with ID: {victimId}")
                else:
                    print(f"{victimId}")

                if killerNames:  # Check if there are killers
                    # Use the last killer as the one who landed the final hit
                    lastKiller = killerNames[-1]
                    print(f"Killed by {lastKiller} with ID: {killerId}")
                    print(f"Timestamp of CHAMPION_KILL: {timestamp}\n")


#getTimestampOfChampionKillWithId(data)


In [21]:
def displayParticipantParameters(data, participantIds):

    # Display the extracted parameter values
    for participantId in participantIds:
        print(f"Participant {participantId} Parameters:")
        for idx, frameData in enumerate(participantFrames):
            damageStats = frameData.get(participantId, {}).get('damageStats', {})
            totalDamageDone = damageStats.get('totalDamageDone', 0)
            totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
            totalDamageTaken = damageStats.get('totalDamageTaken', 0)

            print(f"Frame {idx + 1}:")
            print(f"  Total Damage Done: {totalDamageDone}")
            print(f"  Total Damage Done To Champions: {totalDamageDoneToChampions}")
            print(f"  Total Damage Taken: {totalDamageTaken}")

# Example usage:
# Assuming 'data' is the loaded JSON data
displayParticipantParameters(data, ['2'])


Participant 2 Parameters:
Frame 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 2:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 3:
  Total Damage Done: 3145
  Total Damage Done To Champions: 0
  Total Damage Taken: 949
Frame 4:
  Total Damage Done: 7928
  Total Damage Done To Champions: 0
  Total Damage Taken: 1432
Frame 5:
  Total Damage Done: 11557
  Total Damage Done To Champions: 850
  Total Damage Taken: 2279
Frame 6:
  Total Damage Done: 13829
  Total Damage Done To Champions: 850
  Total Damage Taken: 2391
Frame 7:
  Total Damage Done: 14353
  Total Damage Done To Champions: 1009
  Total Damage Taken: 2676
Frame 8:
  Total Damage Done: 21139
  Total Damage Done To Champions: 1332
  Total Damage Taken: 4437
Frame 9:
  Total Damage Done: 33762
  Total Damage Done To Champions: 1332
  Total Damage Taken: 6021
Frame 10:
  Total Damage Done: 40383
  Total Damage Done To Champions: 1332
  Total Dam

In [22]:
def getParticipantIdByKill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events',[]):
            eventType = event.get('type')
            if eventType == 'CHAMPION_KILL':
                killerId = event.get('killerId')
                print(f"Participant id: {killerId}") 

#getParticipantIdByKill(data)

In [23]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions

    # Print the total damage done to champions only for the specified frames
    print(f"PRE")
    print(f"Frame {startFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"POST")
    print(f"Frame {endFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")

    # Calculate and print the difference
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    print(f"\nTotal Damage Done To Champions Difference: {difference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '2', 7, 8)


PRE
Frame 7:
  Total Damage Done To Champions: 1009
POST
Frame 8:
  Total Damage Done To Champions: 1332

Total Damage Done To Champions Difference: 323


In [38]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0
    totalDamageToChampionsExtraStart = 0
    totalDamageToChampionsExtraEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
            totalDamageToChampionsExtraStart = totalDamageDoneToChampionsExtra
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions
            totalDamageToChampionsExtraEnd = totalDamageDoneToChampionsExtra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Frame {startFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"Frame {startFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraStart}")
    
    print(f"POST")
    print(f"Frame {endFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")
    print(f"Frame {endFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraEnd}")

    # Calculate and print the differences
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    extraDifference = totalDamageToChampionsExtraEnd - totalDamageToChampionsExtraStart
    print(f"\nTotal Damage Done To Champions Difference (Participant {participantId}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extraParticipantId}): {extraDifference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '5', 4, 10)

PRE
Frame 4 - Participant 5:
  Total Damage Done To Champions: 58
Frame 4 - Extra Participant 0:
  Total Damage Done To Champions: 0
POST
Frame 10 - Participant 5:
  Total Damage Done To Champions: 1576
Frame 10 - Extra Participant 0:
  Total Damage Done To Champions: 0

Total Damage Done To Champions Difference (Participant 5): 1518
Total Damage Done To Champions Difference (Extra Participant 0): 0


In [25]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done parameters
    totalDamageDoneStart = 0
    totalDamageDoneEnd = 0
    totalDamageDoneExtraStart = 0
    totalDamageDoneExtraEnd = 0

    totalDamageDoneToChampionsStart = 0
    totalDamageDoneToChampionsEnd = 0
    totalDamageDoneToChampionsExtraStart = 0
    totalDamageDoneToChampionsExtraEnd = 0

    totalDamageTakenStart = 0
    totalDamageTakenEnd = 0
    totalDamageTakenExtraStart = 0
    totalDamageTakenExtraEnd = 0
    
    totalDamageTakenCompared = 0
    totalDamageDoneCompared = 0
    totalDamageDoneToChampionsCompared = 0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneExtra = extraDamageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTakenExtra = extraDamageStats.get('totalDamageTaken', 0)

        if idx + 1 == startFrame:
            totalDamageDoneStart = totalDamageDone
            totalDamageDoneToChampionsStart = totalDamageDoneToChampions
            totalDamageTakenStart = totalDamageTaken
            totalDamageDoneExtraStart = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraStart = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraStart = totalDamageTakenExtra
        elif idx + 1 <= endFrame:
            totalDamageDoneEnd = totalDamageDone
            totalDamageDoneToChampionsEnd = totalDamageDoneToChampions
            totalDamageTakenEnd = totalDamageTaken
            totalDamageDoneExtraEnd = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraEnd = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraEnd = totalDamageTakenExtra
    
    # Print the pre and post data for the first participant
    print(f"Participant {participantId}:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenEnd}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenExtraEnd}")

    # Calculate and print the differences
    damageDoneDifference = totalDamageDoneEnd - totalDamageDoneStart
    extraDamageDoneDifference = totalDamageDoneExtraEnd - totalDamageDoneExtraStart
    damageDoneToChampionsDifference = totalDamageDoneToChampionsEnd - totalDamageDoneToChampionsStart
    extraDamageDoneToChampionsDifference = totalDamageDoneToChampionsExtraEnd - totalDamageDoneToChampionsExtraStart
    damageTakenDifference = totalDamageTakenEnd - totalDamageTakenStart
    extraDamageTakenDifference = totalDamageTakenExtraEnd - totalDamageTakenExtraStart

    print(f"\nTotal Damage Done Difference (Participant {participantId}): {damageDoneDifference}")
    print(f"Total Damage Done Difference (Enemy Laner): {extraDamageDoneDifference}")
    print(f"Total Damage Done To Champions Difference (Participant {participantId}): {damageDoneToChampionsDifference}")
    print(f"Total Damage Done To Champions Difference (Enemy Laner): {extraDamageDoneToChampionsDifference}")
    print(f"Total Damage Taken Difference (Participant {participantId}): {damageTakenDifference}")
    print(f"Total Damage Taken Difference (Enemy Laner): {extraDamageTakenDifference}")
    
    # Calculate differences between participant and enemy
    totalDamageTakenCompared = damageTakenDifference - extraDamageTakenDifference
    totalDamageDoneCompared = damageDoneDifference - extraDamageDoneDifference
    totalDamageDoneToChampionsCompared = damageDoneToChampionsDifference - extraDamageDoneToChampionsDifference
    
    if totalDamageTakenCompared >= 0:
        print(f"\nParticipant {participantId} took {totalDamageTakenCompared} more damage than enemy laner in given timeframe")
    else:
        print(f"\nParticipant {participantId} took {abs(totalDamageTakenCompared)} less damage than enemy laner in given timeframe")

    if totalDamageDoneToChampionsCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneToChampionsCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneToChampionsCompared)}")

    if totalDamageDoneCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneCompared)}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '2', 4, 6)


Participant 2:
  Frame 4 - Total Damage Done: 7928
  Frame 6 - Total Damage Done: 13829
  Frame 4 - Total Damage Done To Champions: 0
  Frame 6 - Total Damage Done To Champions: 850
  Frame 4 - Total Damage Taken: 1432
  Frame 6 - Total Damage Taken: 2391

Enemy Laner:
  Frame 4 - Total Damage Done: 7447
  Frame 6 - Total Damage Done: 15862
  Frame 4 - Total Damage Done To Champions: 579
  Frame 6 - Total Damage Done To Champions: 871
  Frame 4 - Total Damage Taken: 1886
  Frame 6 - Total Damage Taken: 3603

Total Damage Done Difference (Participant 2): 5901
Total Damage Done Difference (Enemy Laner): 8415
Total Damage Done To Champions Difference (Participant 2): 850
Total Damage Done To Champions Difference (Enemy Laner): 292
Total Damage Taken Difference (Participant 2): 959
Total Damage Taken Difference (Enemy Laner): 1717

Participant 2 took 758 less damage than enemy laner in given timeframe

Participant 2 beat enemy laner in given timeframe with total damage done difference of: 

In [27]:
#postGameData

In [28]:
postChallengeData

[{'12AssistStreakCount': 0,
  'abilityUses': 244,
  'acesBefore15Minutes': 0,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 1,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 0,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 956.8693207653203,
  'damageTakenOnTeamPercentage': 0.39941227356877995,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 9,
  'dodgeSkillShotsSmallWindow': 6,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earliestBaron': 1353.9561308999998,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 51,
  'enemyJungleMonsterKills': 0,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMonsterStolenWithoutSmite': 0,
  'firstTurretKilled': 1,
  'firstTurretKilledTime': 772.0563351,
  'flawlessAces': 0,
  '

In [29]:
def getVisionScoreAdvantageLaneOpponent(postData):
    
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges', {})
        visionScoreAdvantageLaneOpponent = challengesData.get('visionScoreAdvantageLaneOpponent', None)

        if visionScoreAdvantageLaneOpponent is not None:
            print(f"Participant ID: {participant['participantId']}, "
                  f"Vision Score Advantage Lane Opponent: {visionScoreAdvantageLaneOpponent}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Vision Score Advantage Lane Opponent data.")
getVisionScoreAdvantageLaneOpponent(postData)

Participant ID: 1, Vision Score Advantage Lane Opponent: -0.517560601234436
Participant ID: 2, Vision Score Advantage Lane Opponent: 0.8316762447357178
Participant ID: 3, Vision Score Advantage Lane Opponent: 1.1093156337738037
Participant ID: 4, Vision Score Advantage Lane Opponent: 0.9751501083374023
Participant ID: 5, Vision Score Advantage Lane Opponent: -0.19174116849899292
Participant ID: 6, Vision Score Advantage Lane Opponent: 1.0727992057800293
Participant ID: 7, Vision Score Advantage Lane Opponent: -0.4540519714355469
Participant ID: 8, Vision Score Advantage Lane Opponent: -0.5259125828742981
Participant ID: 9, Vision Score Advantage Lane Opponent: -0.49370938539505005
Participant ID: 10, Vision Score Advantage Lane Opponent: 0.2372274398803711


In [30]:
def getControlWardTimeCoverageInRiverOrEnemyHalf(postData):
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges',{})
        controlWardTimeCoverageInRiverOrEnemyHalf = challengesData.get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
        
        if controlWardTimeCoverageInRiverOrEnemyHalf is not None:
            print(f"Participant ID: {participant['participantId']}, "
                 f"Control Ward Time Coverage In River or Enemy Half: {controlWardTimeCoverageInRiverOrEnemyHalf}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Data")
            
getControlWardTimeCoverageInRiverOrEnemyHalf(postData)

Participant ID: 1, No Data
Participant ID: 2, Control Ward Time Coverage In River or Enemy Half: 0.8202942721625273
Participant ID: 3, Control Ward Time Coverage In River or Enemy Half: 0.8363244074137197
Participant ID: 4, Control Ward Time Coverage In River or Enemy Half: 0.6348833225708582
Participant ID: 5, No Data
Participant ID: 6, No Data
Participant ID: 7, No Data
Participant ID: 8, No Data
Participant ID: 9, Control Ward Time Coverage In River or Enemy Half: 0.8213240234824767
Participant ID: 10, Control Ward Time Coverage In River or Enemy Half: 0.7564415545136862


In [31]:
def getTakedowns(postData):
    for participant in postData['info']['participants']:
        challengesData = participant.get('challenges',{})
        takedowns = challengesData.get('takedowns', None)
        
        if takedowns is not None:
            print(f"Participant ID: {participant['participantId']}, "
                 f"data: {takedowns}")
        else:
            print(f"Participant ID: {participant['participantId']}, "
                  "No Data")
            
getTakedowns(postData)

Participant ID: 1, data: 12
Participant ID: 2, data: 26
Participant ID: 3, data: 19
Participant ID: 4, data: 22
Participant ID: 5, data: 19
Participant ID: 6, data: 15
Participant ID: 7, data: 19
Participant ID: 8, data: 19
Participant ID: 9, data: 22
Participant ID: 10, data: 24


Relevant data from postGame data:

- visionScoreAdvantageLaneOpponent
- visionScore
- killParticipation
- goldPerMinute
- totalDamageDealtToChampions
- totalDamageTaken
- controlWardTimeCoverageInRiverOrEnemyHalf
- controlWardsPlaced
- soloKills
- stealthWardsPlaced
- wardTakedowns
- teamBaronKills
- teamRiftHeraldKills
- teamElderDragonKills
- dragonKills
- deathsByEnemyChamps
- dragonTakedowns
- kda
- Takedowns
- turretTakedowns
- laneMinionsFirst10Minutes (Maybe?)

In [32]:
def extractParticipantData(participantData, postParticipantData, desiredParameters):
    resultData = {}
    
    for participant in postParticipantData:
        participantId = participant["participantId"]
        championName = participant["championName"]
        puuid = participant["puuid"]
        
        challenges = participant.get("challenges", {})
        participantInfo = {"championName": championName, "puuid": puuid, "data": {}}
        
        for param in desiredParameters:
            if challenges and "challenges" in challenges:
                challengeData = challenges["challenges"]
                if param in challengeData:
                    participantInfo["data"][param] = challengeData[param]
        
        resultData[participantId] = participantInfo

    return resultData

# Example usage:
desiredParametersList = [
    "visionScoreAdvantageLaneOpponent",
    "visionScore",
    "killParticipation",
    "goldPerMinute",
    "totalDamageDealtToChampions",
    "totalDamageTaken",
    "controlWardTimeCoverageInRiverOrEnemyHalf",
    "controlWardsPlaced",
    "soloKills",
    "stealthWardsPlaced",
    "wardTakedowns",
    "teamBaronKills",
    "teamRiftHeraldKills",
    "teamElderDragonKills",
    "dragonKills",
    "deathsByEnemyChamps",
    "dragonTakedowns",
    "kda",
    "takedowns",
    "turretTakedowns",
    "laneMinionsFirst10Minutes"
]

result = extractParticipantData(participantData, postParticipantData['info']['participants'], desiredParametersList)

# Print or further process the extracted data
for participantId, data in result.items():
    print(f"Participant {participantId} ({data['championName']}):")
    for param, value in data["data"].items():
        print(f"{param}: {value}")
    print("\n")


TypeError: list indices must be integers or slices, not str